## Кейс 2

**Шаг 1.** Выгружаем в Google Colab датасет с записями. Диапазон данных взят не 2 недели, как в условии тестового задания, а два полных месяца (с 2016-10-01 по 2016-12-31). На это есть несколько причин. 

Во-первых, при сравнительно коротком периоде выгрузки образуется большое количество частичных сессий (цепочек касаний), в которых отражена лишь финальная транзация без детализации первичных касаний. Следовательно строить Топ-10 самых популярных цепочек будет крайне проблематично.

Во-вторых, даже массив информации за несколько месяцев весит всего несколько десятков МБ, поэтому критической просадки по скорости обработки датафрейма не наблюдается.

In [1]:
query = """select             
                 concat("id",fullVisitorId) as full_visitor_id,
                 visitNumber as visit_number,
                 visitStartTime as visit_start_time,
                 max(date) as dt,
                 replace(replace(concat(max(trafficSource.source),"_",max(trafficSource.medium)),"(",""),")","") as utm,
                 max(hits.transaction.transactionid) as transaction_id
           from `bigquery-public-data.google_analytics_sample.ga_sessions_*` as tbl, unnest(hits) as hits
           where
                (_table_suffix between '20161101' and '20161231')
          group by fullVisitorId, visitNumber, visitStartTime
          order by fullVisitorId, visitStartTime;"""

**Шаг 2.** Загрузить файл csv из Google Drive в Pandas и удалить записи по тем пользователям, кто не совершил ни одной покупки за указанный промежуток времени.

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import copy

In [3]:
parser = lambda date: dt.datetime.strptime(date, '%Y%m%d')

In [4]:
df = pd.read_csv('/content/drive/MyDrive/bq-results-20221124-141536-1669299386744/bq-results-20221124-141536-1669299386744.csv',
                 parse_dates=['dt'], date_parser=parser)

In [5]:
df.head()

,full_visitor_id,visit_number,visit_start_time,dt,utm,transaction_id
0,id0000020424342248747,1,1480578901,2016-11-30,direct_none,NaN
1,id0000174067426171406,1,1478846641,2016-11-10,direct_none,NaN
2,id0000174067426171406,2,1478927157,2016-11-11,direct_none,NaN
3,id000033471059618621,1,1480289088,2016-11-27,youtube.com_referral,NaN
4,id0000388388833742701,1,1480256233,2016-11-27,youtube.com_referral,NaN


In [6]:
df_len = df.shape[0]
print(f'Текущий размер датафрейма - {df_len} записей')

Текущий размер датафрейма - 193096 записей


In [7]:
df['is_transaction'] = df.groupby('full_visitor_id')['transaction_id'].transform(sum)
df_tr = copy.deepcopy(df[df['is_transaction']!=0].loc[:, df.columns != 'is_transaction'])

In [8]:
df_tr_len = df_tr.shape[0]
print(f'Текущий размер датафрейма - {df_tr_len} записей')

Текущий размер датафрейма - 6130 записей


In [9]:
del df

**Шаг 3.** Приступаем к решению "классической задачи" на происк непрерывных сессий в рамках отдельно взятых пользоватей. При будем придерживаться следующих принципов:


*   **Все клики в рамках окна "юзер" должны быть отсортированы в хронологическом порядке**
*   **У пользователя может быть несколько цепочек касаний**
*   **Действия пользователя, которые не укладываются в одну из результативных сессий (цепочек касаний) отбрасываются** 
*   **Полная цепочка касаний должна начинаться с единицы (номер визита) и заканчиваться транзакцией, прочие сессии являются частичными и учитываются отдельно**
*   **Временный лаг между отдельно взятыми кликами не учитывается для простоты расчетов**



In [10]:
df_tr.head()

,full_visitor_id,visit_number,visit_start_time,dt,utm,transaction_id
121,id0007617910709180468,1,1480870581,2016-12-04,direct_none,NaN
122,id0007617910709180468,2,1481052780,2016-12-06,direct_none,NaN
123,id0007617910709180468,3,1481559832,2016-12-12,direct_none,ORD201612123196
365,id0020342483972052451,1,1478351821,2016-11-05,direct_none,NaN
366,id0020342483972052451,2,1481074662,2016-12-06,direct_none,ORD201612063248


In [11]:
df_tr['transaction_id_fillna'] = df_tr.groupby('full_visitor_id')['transaction_id'].fillna(method='bfill')
df_transactions = copy.deepcopy(df_tr[df_tr['transaction_id_fillna'].notna()]
                                .loc[:,['transaction_id_fillna','visit_number','utm']])

In [12]:
del df_tr

In [13]:
df_transactions.head()

,transaction_id_fillna,visit_number,utm
121,ORD201612123196,1,direct_none
122,ORD201612123196,2,direct_none
123,ORD201612123196,3,direct_none
365,ORD201612063248,1,direct_none
366,ORD201612063248,2,direct_none


In [14]:
agg_func_selection = {'visit_number': ['first'], 'utm':[lambda utm: list(utm)]}
df_result = df_transactions.groupby(by=['transaction_id_fillna'], as_index=False).agg(agg_func_selection)
df_result.columns = ['transaction_id', 'first_visit_number','chain_utm']
df_result['is_full_chain'] = np.where(df_result['first_visit_number'] == 1 , 1 , 0)
df_result = df_result.drop(columns=['first_visit_number'], axis=1)
df_result['chain_utm'] = df_result['chain_utm'].apply(lambda x: "->".join(x))

In [15]:
df_result.head()

,transaction_id,chain_utm,is_full_chain
0,ORD201611011521,direct_none,1
1,ORD201611011533,direct_none,0
2,ORD201611011599,direct_none,1
3,ORD201611011621,direct_none,0
4,ORD201611011643,direct_none,1


**Шаг 4.** Визуальный контроль перед подведением итогов

In [16]:
df_transactions[df_transactions['transaction_id_fillna']=='ORD201612123196']

,transaction_id_fillna,visit_number,utm
121,ORD201612123196,1,direct_none
122,ORD201612123196,2,direct_none
123,ORD201612123196,3,direct_none


In [17]:
df_result[df_result['transaction_id']=='ORD201612123196']

,transaction_id,chain_utm,is_full_chain
1497,ORD201612123196,direct_none->direct_none->direct_none,1


**Шаг 5.** Результирующие таблицы по итогам разбора кейса

In [18]:
# Цепочка касаний для каждой транзакции
df_result.head()

,transaction_id,chain_utm,is_full_chain
0,ORD201611011521,direct_none,1
1,ORD201611011533,direct_none,0
2,ORD201611011599,direct_none,1
3,ORD201611011621,direct_none,0
4,ORD201611011643,direct_none,1


In [19]:
# Топ-10 самых популярных цепочек касаний
table = pd.pivot_table(df_result, values='chain_utm', index=['is_full_chain', 'chain_utm'], aggfunc=np.size) \
          .reset_index() \
          .rename(columns = {'transaction_id':'count'}) \
          .sort_values(by=['is_full_chain', 'count'],ascending=[False,False]) \
          .groupby(by=['is_full_chain']).head(10)

table.head(20)

,is_full_chain,chain_utm,count
62,1,direct_none,858
69,1,direct_none->direct_none,385
73,1,direct_none->direct_none->direct_none,142
77,1,direct_none->direct_none->direct_none->direct_...,70
79,1,direct_none->direct_none->direct_none->direct_...,28
82,1,direct_none->direct_none->direct_none->direct_...,26
83,1,direct_none->direct_none->direct_none->direct_...,9
113,1,google_organic,9
54,1,dfa_cpm,8
59,1,dfa_cpm->direct_none,6
